In [1]:
import pandas as pd

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [11]:
train.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [8]:
train['date'].drop_duplicates()

0       2013-01-01
1       2013-01-02
2       2013-01-03
3       2013-01-04
4       2013-01-05
           ...    
1821    2017-12-27
1822    2017-12-28
1823    2017-12-29
1824    2017-12-30
1825    2017-12-31
Name: date, Length: 1826, dtype: object

In [9]:
pd.value_counts(train['sales'])

sales
30     14283
27     14231
26     14212
29     14178
28     14164
       ...  
201        1
214        1
206        1
203        1
202        1
Name: count, Length: 213, dtype: int64

In [7]:
for store in train['store'].drop_duplicates():
    print(f'train store: {store}, items: {", ".join(train.loc[train["store"] == store, "item"].drop_duplicates().astype(str))}')
    print(f'test store: {store}, items: {", ".join(test.loc[test["store"] == store, "item"].drop_duplicates().astype(str))}')

train store: 1, items: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50
test store: 1, items: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50
train store: 2, items: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50
test store: 2, items: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50
train store: 3, items: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 

In [13]:
train.loc[train['store'] == 1, 'date']

0         2013-01-01
1         2013-01-02
2         2013-01-03
3         2013-01-04
4         2013-01-05
             ...    
896561    2017-12-27
896562    2017-12-28
896563    2017-12-29
896564    2017-12-30
896565    2017-12-31
Name: date, Length: 91300, dtype: object

In [14]:
train.describe()

,store,item,sales
count,913000.000000,913000.000000,913000.000000
mean,5.500000,25.500000,52.250287
std,2.872283,14.430878,28.801144
min,1.000000,1.000000,0.000000
25%,3.000000,13.000000,30.000000
50%,5.500000,25.500000,47.000000
75%,8.000000,38.000000,70.000000
max,10.000000,50.000000,231.000000


In [37]:
import numpy as np
min_sales, max_sales = 0, 300
range_ = 10
sales_blocks = dict()
base = ord('A')
for j, i in enumerate(np.arange(min_sales, max_sales, range_)):
    low, high = i, i + range_ - 1
    sales_blocks[(low, high)] = chr(base + j)
    if chr(base + j) == 'Z':
        base = ord('a') - j - 1
sales_blocks[(max_sales, 1000000)] = chr(base + len(sales_blocks))

In [38]:
sales_blocks

{(0, 9): 'A',
 (10, 19): 'B',
 (20, 29): 'C',
 (30, 39): 'D',
 (40, 49): 'E',
 (50, 59): 'F',
 (60, 69): 'G',
 (70, 79): 'H',
 (80, 89): 'I',
 (90, 99): 'J',
 (100, 109): 'K',
 (110, 119): 'L',
 (120, 129): 'M',
 (130, 139): 'N',
 (140, 149): 'O',
 (150, 159): 'P',
 (160, 169): 'Q',
 (170, 179): 'R',
 (180, 189): 'S',
 (190, 199): 'T',
 (200, 209): 'U',
 (210, 219): 'V',
 (220, 229): 'W',
 (230, 239): 'X',
 (240, 249): 'Y',
 (250, 259): 'Z',
 (260, 269): 'a',
 (270, 279): 'b',
 (280, 289): 'c',
 (290, 299): 'd',
 (300, 1000000): 'e'}

In [40]:
def convert_code(sales_blocks, x):
    for k, v in sales_blocks.items():
        if x >= k[0] and x <= k[1]:
            return sales_blocks[k]

train['date_processed'] = list(map(lambda x: x[5:], train['date']))
train['store_processed'] = list(map(lambda x: f'${x}', train['store']))
train['item_processed'] = list(map(lambda x: f'#{x}', train['item']))
train['sales_processed'] = list(map(lambda x: convert_code(sales_blocks, x), train['sales']))
test['date_processed'] = list(map(lambda x: x[5:], test['date']))
test['store_processed'] = list(map(lambda x: f'${x}', test['store']))
test['item_processed'] = list(map(lambda x: f'#{x}', test['item']))
test['sales_processed'] = list(map(lambda x: convert_code(sales_blocks, x), test['sales']))
train.head()


,date,store,item,sales,date_processed,store_processed,item_processed,sales_processed
0,2013-01-01,1,1,13,01-01,$1,#1,B
1,2013-01-02,1,1,11,01-02,$1,#1,B
2,2013-01-03,1,1,14,01-03,$1,#1,B
3,2013-01-04,1,1,13,01-04,$1,#1,B
4,2013-01-05,1,1,10,01-05,$1,#1,B
